In [1]:
import pandas
import sklearn.metrics
import torch
import torch.nn as nn
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import importlib
import src.params as params
import src.model as model
import src.main as main
import src.dataloader as dataloader
importlib.reload(dataloader)
importlib.reload(model)
import torch.nn.functional as F
import src.dataloader as dataloader
import pandas as pd
from pandas import DataFrame

In [2]:
args = params.get_args("../config/system_a.yaml")
data,kpi_name = main.get_cmdb(args)
train,valid,test = main.get_dataset(args)
basic = model.Basic(args, data, kpi_name)

Loading preprocessed data from /home/junetheriver/code/aiops/workdir/system_a/data_pre.pkl
Loaded ground truth from gt_pre.pkl
[140  79  24  39  27  21  23  36]
metric_num: 2152, kpi_num: 121, torch.Size([38880, 2153])


In [4]:
data_dt = dataloader.load_dt_raw(args)

In [3]:
cmdb_set = set([ck[0] for ck in kpi_name])
kpi_set = set([ck[1] for ck in kpi_name])

In [4]:
dfs = []
labels = []
for idx, (x,y) in enumerate(train+valid+test):
    cmdb_id, timestamp = x
    labels.append(np.argmax(y))
    cmdb_idx = basic.cmdb_idx[cmdb_id]
    tensor = model.get_kpi_at_time(basic.data, cmdb_idx, timestamp, 10)
    df = DataFrame(tensor, columns=list(basic.kpi_set))
    df.reset_index(inplace=True)
    df["id"] = pd.Series([idx for i in range(len(df.index))])
    dfs.append(df)
ts = pd.concat(dfs)
y_df = DataFrame(labels, columns=["y"])

In [11]:
y_df.head()

,y
0,4
1,3
2,0
3,1
4,5


In [5]:
cols = list(ts.columns)
cols = [cols[-1]] + cols[:-1]
ts = ts[cols]

In [34]:
ts.to_csv("../../workdir/system_a/ts.csv")

In [6]:
ts = remove_const = ts.loc[:, (ts != ts.iloc[0]).any()]

In [15]:
remove_const.to_csv("../../workdir/system_a/ts.csv")
y_df.to_csv("../../workdir/system_a/y.csv")

In [1]:
from tsfresh import extract_features
extracted_features = extract_features(remove_const, column_id="id", column_sort="index")

NameError: name 'remove_const' is not defined

In [104]:
ts.head(15)

,id,index,system.udp.in_rcvbuf_errors_pct,system.fs.inodes.free,weblogic.basic.heap.size,system.io.wrqm_s,system.io.await,system.io.avg_rq_sz,weblogic.webapp.session_max,system.io.rrqm_s,...,redis.clients.blocked,redis.replication.master_repl_offset,system.io.r_s,haproxy.backend.session.rate,haproxy.frontend.bytes.out_rate,haproxy.backend.session.pct,system.io.w_await,haproxy.backend.response.2xx,system.load.5,haproxy.backend.session.time
0,0,0,0.0,108054392.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.10,0.0
1,0,1,0.0,108054392.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.11,0.0
2,0,2,0.0,108054392.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.11,0.0
3,0,3,0.0,108054392.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.09,0.0
4,0,4,0.0,108054392.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.07,0.0
5,0,5,0.0,108054392.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.06,0.0
6,0,6,0.0,108054392.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.05,0.0
7,0,7,0.0,108054392.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.04,0.0
8,0,8,0.0,108054392.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.05,0.0
9,0,9,0.0,108054392.0,0.0,0.0,0.44,5.18,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.44,0.0,0.04,0.0


In [103]:
test = ts[ts.id == 0].describe()[1:].iloc[:,2:]

In [105]:
des_l = []
for idx in range(ts.id.max()+1):
    des_df = ts[ts.id == idx].describe()[1:].iloc[:,2:]
    des_l.append(des_df)

In [120]:
des_l[14].head()

,system.udp.in_rcvbuf_errors_pct,system.fs.inodes.free,weblogic.basic.heap.size,system.io.wrqm_s,system.io.await,system.io.avg_rq_sz,weblogic.webapp.session_max,system.io.rrqm_s,system.io.rkb_s,redis.mem.used,...,redis.clients.blocked,redis.replication.master_repl_offset,system.io.r_s,haproxy.backend.session.rate,haproxy.frontend.bytes.out_rate,haproxy.backend.session.pct,system.io.w_await,haproxy.backend.response.2xx,system.load.5,haproxy.backend.session.time
mean,0.0,3196067.25,0.0,1.600000,0.017000,1.82200,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,3974.600098,0.0,0.017000,1.0,0.071000,9.300000
std,0.0,0.00,0.0,5.059644,0.053759,5.76167,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,392.952393,0.0,0.053759,0.0,0.032472,0.483046
min,0.0,3196067.00,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,3722.000000,0.0,0.000000,1.0,0.040000,9.000000
25%,0.0,3196067.00,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,3734.000000,0.0,0.000000,1.0,0.042500,9.000000
50%,0.0,3196067.00,0.0,0.000000,0.000000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,3734.000000,0.0,0.000000,1.0,0.060000,9.000000


In [121]:
des_tensor = []
for df in des_l:
    des_tensor.append(torch.tensor(df.values).flatten())
statistical_features = torch.stack(des_tensor)

In [134]:
statistical_features = torch.stack(des_tensor)

In [135]:
statistical_features

tensor([[0.0000e+00, 1.0805e+08, 0.0000e+00,  ..., 0.0000e+00, 1.1000e-01,
         0.0000e+00],
        [0.0000e+00, 3.1838e+06, 0.0000e+00,  ..., 0.0000e+00, 2.1000e-01,
         1.0000e+01],
        [0.0000e+00, 1.0805e+08, 2.0298e+03,  ..., 0.0000e+00, 9.0000e-02,
         0.0000e+00],
        ...,
        [0.0000e+00, 3.1815e+06, 0.0000e+00,  ..., 1.0000e+00, 1.5200e+00,
         7.0000e+00],
        [0.0000e+00, 1.0805e+08, 2.0292e+03,  ..., 0.0000e+00, 1.2000e-01,
         0.0000e+00],
        [5.9290e+01, 1.0805e+08, 2.0330e+03,  ..., 0.0000e+00, 4.9300e+00,
         0.0000e+00]], dtype=torch.float64)

In [136]:
torch.save(statistical_features, "../../workdir/system_a/stat_feature.txt")

In [137]:
test_new = torch.load("../../workdir/system_a/stat_feature.txt")

In [138]:
(test_new[0] == test_new[10]).all()

tensor(False)

In [63]:
from torch.utils.data import DataLoader, Dataset

In [90]:
import src.dataloader as dataloader
from src.dataloader import TensorDataset

ImportError: cannot import name 'TensorDataset' from 'src.dataloader' (/home/junetheriver/code/aiops/src/dataloader.py)

In [81]:
for x in dl:
    print(x.shape)

torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([10, 679])
torch.Size([9, 679])


In [ ]:
def train(model, dataset):
    print(np.bincount([np.argmax(data[1]) for data in dataset]))

    print(model.train())
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=args.lr)
    # optimizer = optim.Adam(model.parameters(), lr=args.lr, weight_decay=0.01)

    for epoch in range(args.epoch):
        start_time = datetime.datetime.now()
        running_loss = 0.0
        for i, data in enumerate(dataset):
            inputs, labels = data
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, torch.tensor(labels).unsqueeze(dim=0))
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        print(f"\r epoch {epoch}, running loss {running_loss}, used time {datetime.datetime.now() - start_time}", end='')
    print("Training finished")

In [91]:
test_new.shape

torch.Size([389, 679])

In [139]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(test_new, y_df["y"].values)

DecisionTreeClassifier()

In [140]:
tree.predict(test_new)

array([4, 3, 0, 1, 5, 0, 3, 0, 1, 1, 1, 4, 7, 3, 1, 7, 0, 1, 0, 5, 1, 1,
       1, 1, 1, 7, 0, 7, 2, 3, 0, 0, 5, 2, 3, 7, 0, 0, 6, 7, 0, 2, 3, 1,
       0, 7, 2, 0, 0, 3, 1, 1, 4, 4, 0, 1, 4, 1, 0, 0, 0, 5, 0, 3, 0, 6,
       1, 0, 0, 4, 3, 0, 3, 0, 6, 3, 1, 1, 3, 1, 6, 1, 0, 0, 0, 4, 5, 0,
       4, 3, 7, 0, 1, 0, 3, 1, 1, 3, 2, 0, 0, 6, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 3, 5, 0, 2, 4, 7, 0, 4, 7, 2, 6, 1, 0, 0, 3, 7, 3, 7, 7, 0, 5,
       1, 1, 7, 7, 0, 4, 0, 7, 0, 1, 1, 1, 3, 0, 0, 1, 0, 2, 1, 1, 1, 1,
       6, 7, 1, 1, 6, 5, 0, 0, 7, 0, 0, 6, 4, 0, 1, 4, 6, 0, 0, 0, 0, 7,
       3, 1, 0, 1, 7, 0, 1, 7, 0, 2, 1, 1, 0, 4, 0, 1, 1, 0, 6, 1, 0, 7,
       5, 3, 1, 3, 1, 0, 3, 0, 1, 0, 0, 1, 0, 0, 3, 0, 4, 4, 0, 4, 0, 4,
       0, 2, 2, 2, 6, 0, 6, 4, 5, 1, 0, 1, 0, 3, 1, 0, 3, 1, 7, 1, 6, 2,
       2, 0, 1, 1, 1, 0, 1, 1, 0, 0, 7, 1, 2, 7, 7, 0, 0, 0, 0, 0, 7, 3,
       0, 2, 1, 5, 2, 0, 0, 0, 3, 1, 0, 2, 6, 1, 5, 0, 0, 0, 4, 0, 0, 3,
       5, 7, 0, 0, 5, 1, 0, 7, 7, 2, 6, 5, 7, 1, 0,

In [102]:
test_new[2]

tensor([0.0000e+00, 1.0805e+08, 0.0000e+00, 0.0000e+00, 4.4000e-02, 5.1800e-01,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 1.0000e-03, 0.0000e+00, 3.7313e+06, 3.5147e+01, 3.7843e+01,
        4.0000e-03, 0.0000e+00, 1.0600e+00, 0.0000e+00, 0.0000e+00, 3.0890e+01,
        1.3150e+04, 4.7206e+01, 1.5380e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 3.1618e+01, 0.0000e+00, 2.9000e+01, 8.6529e+07, 0.0000e+00,
        6.0200e-01, 3.5648e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e-02,
        7.0000e+00, 1.0284e+04, 1.0600e+00, 0.0000e+00, 5.8678e+01, 0.0000e+00,
        0.0000e+00, 8.0005e+04, 0.0000e+00, 0.0000e+00, 1.3500e+00, 0.0000e+00,
        6.7655e+07, 0.0000e+00, 9.9307e+01, 0.0000e+00, 6.1000e+00, 4.8000e-02,
        6.0000e-02, 1.8862e+04, 6.9300e-01, 1.0122e+04, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 1.1900e+02, 3.6200e-01, 0.0000e+00, 0.0000e+00, 2.2123e+04,
        8.0000e-03, 2.0000e-03, 6.2214e+